# Dataprep para o Case da First Decision/ANATEL

Todo o meu código é escrito em inglês (inclusive nomes de variáveis, nomes de métodos, classes, etc.) isso elimina o "portuglês" que deriva de se usar uma linguagem com palavras reservadas em inglês e variáveis em português.

Claro que, de acordo com as necessidades do local onde trabalho, esse padrão pode ser mudado.

## 1. Setup

In [1]:
import re
import pandas as pd
import io
from tqdm import tqdm
from pipe import Pipe

## 2. Aquisição dos Dados

A planilha original (.xlsm) não se encontra propriamente organizada em colunas, com detalhes da organização dos dados descritos abaixo:

 * A coluna "A" contendo a maior parte dos dados
 * Os dados estão separados por vírgula
 * Alguns dos nomes de empresas possuem vírgulas
 * Os textos descritivos de cada empresa estão entre aspas

 Com isso, pode-se elaborar uma estratégia de formatação dos dados.

 Inicialmente, os dados são salvos no Excel como texto simples (.txt), deixando para este notebook a tarefa de formatar os dados.

 Isso facilita, também a repetibilidade dos procedimentos de formatação.
 

In [2]:
with open('Dados\\dataset.txt', 'r', encoding='utf-16') as file:
    text_data = file.readlines()

In [3]:
text_data[:5]

['"name,description,employees,total_funding,city,subcountry,lat,lng"\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n',
 '"fitin,""fitin is the latest fit tech platform that provides a professional marketing and booking tool for gyms, studios, independent instructors and personal trainers. customers want a combination of \'trip advisor\' and \'groupon\', where they can read reviews, check out schedules, browse photos, and bookâ€¦all in one place. gyms and trainers want to be able to fill their classes and schedules. at fitin, we want to connect them and provide the community with a self-sustaining resource that helps everyone. with the wide variety of fitness entrepreneurs out there, we have amazing diversity in our workout options, but they are currently too difficult to find, if you donâ€™t already know where to go â€“ you need to search for hours, reach out to friends and family for referrals that may or may not be a good fit for you. we have over 700 classes already listed in our platform, an

## 3. Limpeza e Shaping dos Dados

### 3.1. Correção do CSV Mal-formado

A correção do CSV tem como centro um loop que passa por todas as linhas do arquivo TXT gerado a partir da planilha (exportando pelo próprio Excel), registrando todos os erros encontrados e permitindo que problemas nas linhas sejam detectados.

A partir do loop, existem testes que efetivamente detectam os problemas, checagens visuais e procedimentos de correção (obviamente, os procedimentos de correção estão postos em células que precedem o loop, de forma que um problema corrigido deixa de aparecer na checagem).

O processo de limpeza termina quando todos os testes ficam positivos e a inspeção visual não fornece resultados positivos para problemas.

A partir do fim da limpeza dos dados, resolvem-se problemas de shaping.

In [4]:
@Pipe
def subst(text:str, pattern, replacement, count=0, way='regex'):

    if way == 'regex':
        result = re.sub(pattern, replacement, text, count)
    elif way == 'replace':
        result = text.replace(pattern, replacement, -1 if count == 0 else count)
    else:
        raise ValueError(f'"{way}"? This is not the way.')

    return result

In [5]:
def correct_formatting(original_line:str):
    result = original_line.strip()

    result = (result
        | subst('\t"', '', way='replace')
        | subst('"\t', '', way='replace')
        | subst('\t+', ' ')
        | subst(' +', ' ')
        | subst(',\"+', '","', count=1)
        | subst('\"+,', '\",')
        | subst(f'[^a-zA-Z0-9",. /\_]', ' ')
        | subst('(?<!,)"(?!,)', '')
        | subst('\"\t+\"','')
        | subst('"+,', ' ",')
    )

    return result

In [6]:
text_data = [correct_formatting(line) for line in text_data]

In [7]:
header = '"' + '","'.join(text_data[0].split(',')) + '"'
header

'"name","description","employees","total_funding","city","subcountry","lat","lng"'

In [8]:
text_data[1]

'fitin ","fitin is the latest fit tech platform that provides a professional marketing and booking tool for gyms, studios, independent instructors and personal trainers. customers want a combination of  trip advisor  and  groupon , where they can read reviews, check out schedules, browse photos, and book   all in one place. gyms and trainers want to be able to fill their classes and schedules. at fitin, we want to connect them and provide the community with a self sustaining resource that helps everyone. with the wide variety of fitness entrepreneurs out there, we have amazing diversity in our workout options, but they are currently too difficult to find, if you don   t already know where to go     you need to search for hours, reach out to friends and family for referrals that may or may not be a good fit for you. we have over 700 classes already listed in our platform, and we want to be the resource that inspires canadians to just get moving   no matter what kind of class that may be

In [9]:
# Esse é o loop que corre pelos dados alimentando uma lista de problemas e um dataframe que contém os dados.
# O dataframe gerado é o que passa pelos testes e inspeções visuais.

problems = []

indices_col = []
names_col = []
descriptions_col = []
employees_col = []
fundings_col = []
cities_col = []
subcountries_col = []
latitudes_col = []
longitudes_col = []
extracted_col = []

check_ndx = 20
check = False

for i in tqdm(range(1, len(text_data))):
#for i in tqdm(range(1, 10)):
    line = text_data[i]

    separator = ','
    quoting_enabled = True
    
    if i in [142, 2627, 3269, 11359, 11533, 11651, 17865]: # Linhas com problemas únicos ou pouco comuns que, por não fazerem parte do target, serão apenas desconsideradas. Esses números de linha não mudam, pois vem do arquivo exportado do .xlsm, que eu não vou tocar.
        continue

    try:
        line = line.strip()

        if check and i == check_ndx:
            print(f'BEFORE >> {line}')

        test_line = line[1:]
        if not (',"' in test_line): # Solução para o problema simples de linhas de CSV em estilo quoted/non-quoted (o primeiro caractere é forçado a ser sempre um quote mark)
            line = test_line.replace('"', '')
            quoting_enabled = False

        if check and i == check_ndx:
            print(f'AFTER >> {line}')
            print(f'QUOTED >> {quoting_enabled}')

        text = '\n'.join([header] + [line])

        if check and i == check_ndx:
            print(f'TEXT >> \n{text}')

        stream = io.StringIO(text)

        extracted = pd.read_csv(stream, sep=separator, quoting=quoting_enabled, quotechar='"').to_dict('list') # Converter o dado em DataFrame ajuda a garantir ao menos que a linha do CSV está bem formada.

        if check and i == check_ndx:
            print(f'EXTRACTED >> {extracted}')

    except Exception as err:
        problem = {
            'row': i
            ,'line': line
            ,'full_line': text
            ,'message': err
            ,'extracted': extracted
        }

        problems.append(problem)

        continue
    
    indices_col.append(i)
    names_col.append(extracted['name'][0])
    descriptions_col.append(extracted['description'][0])
    employees_col.append(extracted['employees'][0])
    fundings_col.append(extracted['total_funding'][0])
    cities_col.append(extracted['city'][0])
    subcountries_col.append(extracted['subcountry'][0])
    latitudes_col.append(extracted['lat'][0])
    longitudes_col.append(extracted['lng'][0])
    extracted_col.append(extracted) # Essa coluna temporária contém o dado em texto do qual foi criada cada linha e serve como referência para a detecção da causa raíz de cada problema.

prob = pd.DataFrame(problems)
prob.head()

100%|██████████| 21299/21299 [00:33<00:00, 627.73it/s]


""


In [10]:
if len(prob) > 0:
    prob.loc[0] # Para ajudar a visualizar o primeiro problema.

In [11]:
data = pd.DataFrame()

data['Index'] = indices_col
data['Name'] = names_col
data['Description'] = descriptions_col
data['Employees'] = employees_col
data['Funding'] = fundings_col
data['City'] = cities_col
data['Subcountry'] = subcountries_col
data['Latitude'] = latitudes_col
data['Longitude'] = longitudes_col
data['Extracted'] = extracted_col

data.head()

,Index,Name,Description,Employees,Funding,City,Subcountry,Latitude,Longitude,Extracted
0,1,"fitin """,fitin is the latest fit tech platform that pro...,1.0,NaN,toronto,ontario,43.70011,79.416300,"{'name': ['fitin ""'], 'description': ['fitin i..."
1,2,"quinditech """,our services include independent collaborati...,10.0,NaN,montreal,quebec,45.50884,73.587810,"{'name': ['quinditech ""'], 'description': ['ou..."
2,3,"botfirst """,chatlanding is a digital chatbot platform wher...,1.0,NaN,toronto,ontario,43.70011,79.416300,"{'name': ['botfirst ""'], 'description': ['chat..."
3,4,ravver,ai powered army of health professionals in you...,1.0,NaN,quebec,quebec,46.82596,71.235223,"{'name': ['ravver'], 'description': ['ai power..."
4,5,"stylify """,stylify is a mobile platform that connects peo...,1.0,NaN,toronto,ontario,43.70011,79.416300,"{'name': ['stylify ""'], 'description': ['styli..."


In [12]:
data.loc[9147, 'Description']

'n  97 is an e commerce store that provides customers with looks inspired off the runway with an affordable price tag.'

### 3.2. Correção das Colunas Numéricas

Aqui temos testes automatizados que detectam problemas na conversão de dados numéricos para seus tipos adequados.
As linhas que aparecem no dataframe de teste são as que tem problemas, portanto um dataframe vazio é um teste bem-suscedido.

In [13]:
data['test'] = pd.to_numeric(data['Employees'], errors='coerce')
data[(data['test'].isna()) & (data['Employees'].notna())]

,Index,Name,Description,Employees,Funding,City,Subcountry,Latitude,Longitude,Extracted,test


In [14]:
data['Funding'] = data['Funding'].str.replace('"', '')
data['Funding'] = data['Funding'].str.replace(',', '')


data.loc[data['Funding'] == ' ','Funding'] = None
data['Funding'] = data['Funding'].astype(float)

In [15]:
data['test'] = pd.to_numeric(data['Funding'], errors='coerce')
data[(data['test'].isna()) & (data['Funding'].notna()) & (data['Funding'] != ' ')]

,Index,Name,Description,Employees,Funding,City,Subcountry,Latitude,Longitude,Extracted,test


In [16]:
data['test'] = pd.to_numeric(data['Latitude'], errors='coerce')
data[(data['test'].isna()) & (data['Latitude'].notna())]

,Index,Name,Description,Employees,Funding,City,Subcountry,Latitude,Longitude,Extracted,test


In [17]:
data['test'] = pd.to_numeric(data['Longitude'], errors='coerce')
data[(data['test'].isna()) & (data['Longitude'].notna())]

,Index,Name,Description,Employees,Funding,City,Subcountry,Latitude,Longitude,Extracted,test


In [18]:
data.drop(columns='test', inplace=True)

### 3.3. Correção das Colunas de Texto

As colunas de texto são limpas a partir de inspeção visual, usando o "scrollable element" sempre que necessário.

In [19]:
data['Name'] = data['Name'].str.replace('"', '')
data['Name'] = data['Name'].str.replace('_', '')

In [20]:
list(data['Name'])

['fitin ',
 'quinditech ',
 'botfirst ',
 'ravver',
 'stylify ',
 'peloton technologies ',
 'weave travels ',
 'space   york university',
 'glaucus.ai ',
 'magnifi ',
 'eara technologies ',
 'amika   homecare ',
 'alitheia technologies ',
 'justgotthat  ',
 'bundle rewards ',
 'nugget.ai ',
 'titanfile ',
 'candor ai ',
 'velocia ',
 'soundpays ',
 'market iq ',
 'valtools company',
 'n via education',
 'trustunion ',
 'passionfruit ',
 'arz',
 'the original smiths ',
 'zighra ',
 'edify learning spaces ',
 'jauntin  ',
 'mapyourproperty ',
 'motive health ',
 'thornhill medical ',
 'way2go apps inc.  ',
 'thedatabridge.org ',
 'precision biomonitoring ',
 'testtoad ',
 'west immigration consulting ',
 'leo prestte ',
 'locatemotion ',
 'ittelstand finanical',
 'fleeky ',
 'coiniq ',
 'izademy.com',
 'learnademy.com ',
 'buddi ',
 'modenala automotive sp ',
 'rally ',
 'sitwidse0ul by h y ',
 'roluton depot 250 mg',
 'giver.be ',
 'abanban ',
 'ntrepreneurship ubc',
 'ophelia financial

In [21]:
data['City'].unique()

array(['toronto', 'montreal', 'quebec', 'victoria', 'markham', 'kelowna',
       'vancouver', 'waterloo', 'ottawa', 'calgary', 'guelph', 'winnipeg',
       'burlington', 'halifax', 'edmonton', 'vaughan', 'whitehorse',
       'london', 'fredericton', 'chicoutimi', 'kitchener', 'sudbury',
       'mississauga', 'north york', 'richmond hill', 'brampton',
       'abbotsford', 'charlottetown', 'galiano island', 'sherbrooke',
       'kingston', 'beaumont', 'sylvan lake', 'richmond', 'barrie',
       'st. john s', 'milton', 'laval', 'sherwood park', 'cranbrook',
       'oshawa', 'saint jean sur richelieu', 'cornwall', 'saskatoon',
       'cambridge', 'saint john', 'boston creek', 'nelson', 'cochin',
       'princeville', 'etobicoke', 'langley', 'blainville', 'scarborough',
       'oakville', 'strathroy', 'longueuil', 'regina',
       'whitchurch   stouffville', nan, 'coquitlam', 'surrey',
       'niagara falls', 'kamloops', 'okotoks', 'duncan', 'chatsworth',
       'tecumseh', 'west vancouver'

In [22]:
data['Subcountry'].unique()

array(['ontario', 'quebec', 'british columbia', 'alberta', 'manitoba',
       'nova scotia', 'yukon', 'new brunswick', 'prince edward island',
       'newfoundland and labrador', 'saskatchewan', nan,
       'northwest territories'], dtype=object)

In [23]:
data = data[data['Name'].notna()]
data.drop(columns='Extracted', inplace=True)

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21289 entries, 0 to 21291
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Index        21289 non-null  int64  
 1   Name         21289 non-null  object 
 2   Description  21289 non-null  object 
 3   Employees    21289 non-null  float64
 4   Funding      10 non-null     float64
 5   City         21276 non-null  object 
 6   Subcountry   21276 non-null  object 
 7   Latitude     21276 non-null  float64
 8   Longitude    21276 non-null  float64
dtypes: float64(4), int64(1), object(4)
memory usage: 1.6+ MB


## 4. Gravação dos Dados Limpos

In [25]:
data.to_csv('Dados\\clean_data.csv')